In [ ]:
import cv2
import numpy as np
import math

def findLetters(binary, path):
	inverted = cv2.bitwise_not(binary)

	#Detect centers of letters or their parts
	contours,hierarchy = cv2.findContours(inverted, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	centers = []
	for contour in contours:
		x,y,w,h = cv2.boundingRect(contour)
		centers.append([int(x+w/2),int(y+h/2)])
	centers.sort(key = lambda c: c[0])
	
	#Join some of the broken letters (only leters broken by horizontal gaps are joined)
	for i in range(1,len(centers)):
		x1,y1 = centers[i-1]
		x2,y2 = centers[i]
		theta = math.atan2(y2-y1, x2-x1)*180/math.pi
		if abs(abs(theta)-90)<=20:
			cv2.line(inverted, (x1,y1), (x2,y2), 255, 2)

	#Detect letters again
	rectangles = []
	contours,hierarchy = cv2.findContours(inverted, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	for contour in contours:
		x,y,w,h = cv2.boundingRect(contour)
		rectangles.append([x,w])

	#Scan and store letters in a folder from left to right
	i=1
	rectangles.sort(key = lambda r: r[0])
	for rectangle in rectangles:
		x,w = rectangle
		letter = binary[:,x:x+w].copy()
		cv2.imwrite(path + "/letter%d.jpg" %i, letter)
		i+=1